In [46]:
import requests
import pandas as pd

from glob import glob

In [39]:
r = requests.get('https://www.americashealthrankings.org/graphql?operationName=StateDataByEdition&variables=%7B%22editionId%22%3A182%2C%22metricTypeId%22%3A118%7D&extensions=%7B%22persistedQuery%22%3A%7B%22version%22%3A1%2C%22sha256Hash%22%3A%226293743e4fa58668f7bf57e2875477b6c40bb13f93e6c8e1810029183204409a%22%7D%7D')

In [40]:
data = r.json()
# data["data"]["report"]["reportMeasures"]

In [43]:
df_a = pd.DataFrame(data["data"]["report"]["reportMeasures"][0]["measure"]["metricData"]).sort_values("displayYear")
df_a = pd.concat([df_a, pd.json_normalize(df_a["state"])], axis=1)
df_a = df_a[["stateName", "displayYear", "value"]]
df_a.sort_values("displayYear")

,stateName,displayYear,value
1454,Alaska,1990,0.471
1452,North Dakota,1990,0.392
1451,Nebraska,1990,0.379
1450,New Hampshire,1990,0.437
1449,New Jersey,1990,0.369
...,...,...,...
3,Massachusetts,2017,0.473
2,Michigan,2017,0.482
24,Mississippi,2017,0.462
25,Missouri,2017,0.479


In [52]:
df_a = pd.read_excel("https://www.shsu.edu/eco_mwf/Frank_Gini_2018.xls")
df_a = df_a[["Year", "State", "Gini"]]
df_a.columns = [x.lower() for x in df_a.columns]
df_a["state"] = df_a["state"].str.upper()
df_a

,year,state,gini
0,1917,UNITED STATES,0.507681
1,1917,ALABAMA,0.399792
2,1917,ARIZONA,0.399658
3,1917,ARKANSAS,0.376483
4,1917,CALIFORNIA,0.449363
...,...,...,...
5215,2018,VIRGINIA,0.597394
5216,2018,WASHINGTON,0.604622
5217,2018,WEST VIRGINIA,0.545044
5218,2018,WISCONSIN,0.577300


In [57]:
output = []
for file in glob("data_external/United States/ACSDT5Y*.B19083-*"):
    year = file[35:39]
    df_tmp = pd.read_csv(file)
    df_tmp["year"] = year
    output.append(df_tmp)

df = pd.concat(output, ignore_index=True)
output = []
i = 0
while 3*i < df.shape[0]:
    data = df.iloc[3*i:3*i + 3]

    state = data.iloc[0]["Label (Grouping)"]
    year = data.iloc[0]["year"]
    value = data.iloc[1]["Gini Index"]
    moe = data.iloc[2]["Gini Index"]

    output.append({
        "state": state.upper(),
        "gini": value,
        "moe": moe,
        "year": year
    })
    i += 1

df_b = pd.DataFrame(output)
df_b["year"] = df_b["year"].astype(int)
df_b

,state,gini,moe,year
0,ALABAMA,0.470,±0.003,2010
1,ALASKA,0.412,±0.006,2010
2,ARIZONA,0.453,±0.002,2010
3,ARKANSAS,0.459,±0.003,2010
4,CALIFORNIA,0.469,±0.001,2010
...,...,...,...,...
567,WASHINGTON,0.4574,±0.0020,2020
568,WEST VIRGINIA,0.4661,±0.0031,2020
569,WISCONSIN,0.4448,±0.0019,2020
570,WYOMING,0.4393,±0.0079,2020


In [60]:
df = pd.concat([df_a[df_a["year"] < 2010], df_b], ignore_index=True)
df.to_csv("data_curated/United States/Income_Inequality.csv", index=False)